In [3]:
import cv2
import torch
import time
from pathlib import Path

def convert_to_roi_tuple(roi):
    x_min = min(point[0] for point in roi)
    y_min = min(point[1] for point in roi)
    x_max = max(point[0] for point in roi)
    y_max = max(point[1] for point in roi)
    return (x_min, y_min, x_max, y_max)

def is_inside_roi(box, roi):
    x_min, y_min, x_max, y_max = box
    roi_x_min, roi_y_min, roi_x_max, roi_y_max = roi
    return (x_min >= roi_x_min and y_min >= roi_y_min and x_max <= roi_x_max and y_max <= roi_y_max)

def check_card_consistency(cards):
    main_cards = {card for card in cards if card.isdigit() or card in ['j', 'q', 'k', 'a']}
    for main_card in main_cards:
        card_corners = [f'{main_card}g', f'{main_card}d']
        if any(corner in cards for corner in card_corners):
            return main_card
    return None

def is_corner_inside_main_card(corner_box, main_card_box):
    """Sprawdza, czy róg karty (corner_box) znajduje się wewnątrz głównej karty (main_card_box)."""
    corner_x_min, corner_y_min, corner_x_max, corner_y_max = corner_box
    main_x_min, main_y_min, main_x_max, main_y_max = main_card_box

    return (corner_x_min >= main_x_min and corner_y_min >= main_y_min and 
            corner_x_max <= main_x_max and corner_y_max <= main_y_max)

def calculate_score(cards):
    values = {'j': 10, 'q': 10, 'k': 10, 'a': 11}
    score = 0
    aces = 0
    for card in cards:
        if card in values:
            score += values[card]
        else:
            score += int(card)
        if card == 'a':
            aces += 1
    while score > 21 and aces:
        score -= 10
        aces -= 1
    return score

def determine_result(players_scores, dealer_score):
    results = {}
    for player, score in players_scores.items():
        if score == 21 and len(players_scores[player]) == 2:
            results[player] = "BLACKJACK"
        elif score > 21:
            results[player] = "za dużo"
        else:
            if dealer_score > 21 or (score <= 21 and score > dealer_score):
                results[player] = "wygrana"
            elif score == dealer_score:
                results[player] = "remis"
            else:
                results[player] = "przegrana"
    return results

# Definiowanie ROI
roi_player1 = convert_to_roi_tuple([(200, 70), (368, 108), (78, 236), (266, 294)])
roi_player2 = convert_to_roi_tuple([(368, 108), (544, 147), (266, 294), (471, 355)])
roi_player3 = convert_to_roi_tuple([(544, 147), (771, 202), (471, 355), (755, 444)])
roi_dealer = convert_to_roi_tuple([(68, 251), (753, 466), (2, 354), (695, 594)])

# Inicjalizacja
weights_path = Path('path/to/weights') #uzupełnić
droidcam_url = "http://url/video" #uzupełnić
model = torch.hub.load('ultralytics/yolov5', 'custom', path=weights_path, force_reload=True)
wait_time = 5

while True:
    cap = cv2.VideoCapture(droidcam_url)
    ret, frame = cap.read()
    if not ret:
        print("Nie udało się uzyskać obrazu z kamery.")
        cap.release()
        break
    frame = cv2.resize(frame, (800, 600))
    results = model(frame)
    results.render()


    player_cards = {'player1': [], 'player2': [], 'player3': [], 'dealer': []}
    for *xyxy, conf, cls in results.xyxy[0]:
        box = [int(x) for x in xyxy]
        card_label = model.names[int(cls)]
        for player, roi in [('player1', roi_player1), ('player2', roi_player2),
                            ('player3', roi_player3), ('dealer', roi_dealer)]:
            if is_inside_roi(box, roi):
                player_cards[player].append((box, card_label))
                break

    players_scores = {}
    for player, cards in player_cards.items():
        identified_cards = []
        for main_box, main_label in cards:
            if main_label.isdigit() or main_label in ['j', 'q', 'k', 'a']:
                valid_corners = [corner_label for corner_box, corner_label in cards if corner_label.startswith(main_label) and is_corner_inside_main_card(corner_box, main_box)]
                if valid_corners:
                    identified_cards.append(main_label)
        score = calculate_score(identified_cards)
        players_scores[player] = score
        if identified_cards:
            result_str = ', '.join(identified_cards) + " - wynik: " + ("BLACKJACK" if score == 21 and len(identified_cards) == 2 else str(score))
            if score > 21:
                result_str += ", za dużo"
            print(f"{player.capitalize()} posiada karty: {result_str}")

    dealer_score = players_scores.pop('dealer')
    game_results = determine_result(players_scores, dealer_score)
    for player, result in game_results.items():
        print(f"{player.capitalize()}: {result}")

    cv2.imshow('YOLOv5 Detections', frame)
    
    key = cv2.waitKey(0)
    if key == ord('q'):
        cap.release()
        break
    elif key == ord('c'):
        print("Następna detekcja:")
        cap.release()
        time.sleep(wait_time)
        continue

cap.release()
cv2.destroyAllWindows()

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\OsiBook/.cache\torch\hub\master.zip
YOLOv5  2023-12-2 Python-3.8.18 torch-2.1.1 CPU

Fusing layers... 
Model summary: 267 layers, 46312908 parameters, 0 gradients, 108.3 GFLOPs
Adding AutoShape... 


Player1 posiada karty: j, 6 - wynik: 16
Player2 posiada karty: a, 9 - wynik: 20
Player3 posiada karty: 6, 5, 4 - wynik: 15
Dealer posiada karty: 6, 2, 4, 6 - wynik: 18
Player1: przegrana
Player2: wygrana
Player3: przegrana
Następna detekcja:
Player1 posiada karty: j, 6 - wynik: 16
Player2 posiada karty: a, 9 - wynik: 20
Player3 posiada karty: 6, 5, 4 - wynik: 15
Dealer posiada karty: 6, 2, 4, 6 - wynik: 18
Player1: przegrana
Player2: wygrana
Player3: przegrana
